In [1]:
# import ui as ui
import win32com.client
import pythoncom
import os

import pandas as pd
import numpy as np
import time

# from PyQt5 import QtCore, QtWidgets

import logging

In [2]:
app = win32com.client.Dispatch("AutoCAD.Application")
doc = app.ActiveDocument
model = doc.ModelSpace

In [3]:
ls = []
for ob in model:
    ls.append([ob.Handle, ob.EntityName])
df_ent_type = pd.DataFrame(ls, columns=['handle', 'entity_name'])

In [4]:
df = df_ent_type[df_ent_type['entity_name']=='AcDbTable']

In [5]:
df

,handle,entity_name
1,10C25,AcDbTable
12,11838,AcDbTable


In [11]:
b1 = model.Database.HandleToObject(df.iloc[1,0])

In [38]:
ls = []
bm = ''
sm = ''
for r in range(2, b1.Rows):
    em = b1.GetCellValue(r, 4)
    if em is not None:
        bm1 = b1.GetCellValue(r, 2)
        if bm1 is not None:
            bm = bm1


        if em[:2] == 'До':
            em1 = em[14:]
            em2 = 'До'
        if em[:2] == 'За':
            em1 = em[13:]
            em2 = 'За'
        if em[:2] == 'На':
            em1 = em[9:]
            em2 = 'На'


        sm = b1.GetCellValue(r, 5)
        if sm is None:
            sm = ''

        ls.append([bm,em1,em2,sm])

df1 = pd.DataFrame(ls, columns=['bm', 'em', 'em2', 'sm'])
df1

,bm,em1,em2,sm
0,Мн,М1,До,
1,Мз,М7,До,
2,М1,М203,До,"-1/3, -5"
3,М1,М11,До,"-1/3, +5"
4,М1,М13,До,+1/3
...,...,...,...,...
289,М216,М218,За,-212
290,М216,М206,До,+212
291,М218,201П,На,-204/206
292,М218,202П,На,"+204/206, +208"


In [34]:
s1 = 'На колію 1 АП'
s1[9:]

'1 АП'

In [2]:
# логирование
logging.basicConfig(filename='log.txt', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
# logging.debug('Start of program')
# logging.info('So should this')
# logging.warning('And this, too')

In [3]:
class Acad:
    def __init__(self):





        self.app = None
        self.doc = None
        self.model = None

        self.ls_contact = ['1', '2', '3', '4', '11', '12', '13', '21', '22', '23', '31', '32', '33', '41', '42', '43',
                           '51', '52', '53', '61', '62', '63', '71', '72', '73', '81', '82', '83', '111', '112', '113',
                           '121', '122', '123']
        self.ds_st_b = {
            'df_101': ['В19', 'В110', 'В111', 'В112', 'В29', 'В210', 'В211', 'В212', 'В18', 'В28', ],
            'df_C': ['В16', 'В17', 'В26', 'В27', 'В25'],
            'df_contacts': ['В18', 'В28'],
            'df_bok': ['Б9', 'Б10', 'Б11', 'Б12', 'Б4', 'Б3', 'Б2', 'Б1'],
        }
        self.ds_st_v_row_col = {
                            '1': (1, 0),
                            '2': (1, 2),
                            '3': (3, 0),
                            '4': (3, 2),
                            '5': (5, 0),
                            '6': (5, 2),
                            '7': (7, 0),
                            '8': (7, 2),
                            '9': (9, 0),
                            '10': (9, 2),
                            '11': (11, 0),
                            '12': (11, 2),
                            '13': (13, 0),
                            '14': (13, 2),
                            '15': (15, 0),
                            '16': (15, 2),
                            '17': (17, 0),
                            '18': (17, 2),
                            '19': (19, 0),
                            '20': (19, 2),
                        }
        self.df_st_shina_montag = pd.DataFrame(columns=['st', 'sh', 'loc', 'contact', 'polka'])
        self.ls_st_name = ['1', '2', '3']
        self.ls_st_sh_name = ['М', 'П', 'МСХ', 'СХМ']


        self.docs_open = {}
        self.circuit_ls_name = None
        self.montag_ds_name = None

        self._init_docs()
        self._init_acad()

        self.get_docs()

        self.df_ent_type = None
        self.df_B = None
        self.df_C = None
        self.df_101 = None
        self.df_st_b = None

        self.df_poly = None
        self.df_line = None
        self.df_circle = None
        self.df_mtext = None
        self.df_text = None
        self.df_table = None

        self.df_st_table = None
        self.df_st_v_table = None
        self.df_st_sh_table = None
        self.df_sx_contacts = None
        self.df_sx_sh_circle = None

    def _init_acad(self):
        self.app = win32com.client.Dispatch("AutoCAD.Application")
        if not self.app.Visible:
            self.app.Visible = True

        # for doc in self.app.Documents:
        #     doc.Close(False)

        # self.load_docs()

    def _init(self):
        self.doc = self.app.ActiveDocument
        self.model = self.doc.ModelSpace

    def _set_doc(self, name_sxema):
        try:
            if name_sxema in self.docs_open:
                self.doc = self.docs_open[name_sxema]
                self.model = self.doc.ModelSpace
            else:
                self.doc = self.open_doc(name_sxema)
                time.sleep(2)

            self.load_entity_from_csv()
            # time.sleep(2)
        except Exception as e:
            logging.error(f'Попытка установить документ {name_sxema} - неудачно {e}')


    def _init_docs(self):
        self.montag_ds_name = {
            '1': 'D:\\Фабричная\\Фабричная\\Монтажные схемы\\2517-АТР.2 л.18 Статив 1.dwg',
            '2': 'D:\\Фабричная\\Фабричная\\Монтажные схемы\\2517-АТР.2 л.19 Статив 2.dwg',
            '3':  'D:\\Фабричная\\Фабричная\\Монтажные схемы\\2517-АТР.2 л.20 Статив 3.dwg',
        }

        self.circuit_ls_name = [
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.4 Схемы РЦ Чет..dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.9 Переезды.dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.10 Схемы увязок.dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.11 Ограждение.dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.12 Загальні схеми.dwg',
             'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.13 Контр.предохр..dwg'
        ]


    def add_circle(self, center, radius):
        self.model.AddCircle(self.a_point(*center), radius)


    def read_entity_to_csv_all_docs(self):
        self.get_docs()
        for doc in self.docs_open.values():
            self.doc = doc
            self.model = self.doc.ModelSpace
            self._read_ent_type()
            self.read_entity_to_csv()


    def read_entity_to_csv(self):
        self._read_polylines()
        self._read_lines()
        self._read_circles()
        self._read_mtexts()
        self._read_texts()
        self._read_tables()

    def _read_polylines(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbPolyline']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'length': b1.Length,
                'x1': round(b1.Coordinates[0], 2),
                'y1': round(b1.Coordinates[1], 2),
                'x2': round(b1.Coordinates[-2], 2),
                'y2': round(b1.Coordinates[-1], 2),
                'points': b1.Coordinates,
                'layer': b1.Layer,
                'color': b1.color,
                'linetype': b1.Linetype,
                'closed': b1.Closed,
                'area': b1.Area,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
                'thickness': b1.Thickness,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='polylines', ext='.csv', folder='csv/'), index=False, sep='$')

    def _read_lines(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbLine']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'length': b1.Length,
                'x1': round(b1.StartPoint[0], 2),
                'y1': round(b1.StartPoint[1], 2),
                'x2': round(b1.EndPoint[0], 2),
                'y2': round(b1.EndPoint[1], 2),
                'layer': b1.Layer,
                'color': b1.color,
                'linetype': b1.Linetype,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
                'thickness': b1.Thickness,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='lines', ext='.csv', folder='csv/'), index=False, sep='$')

    def _read_circles(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbCircle']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'r': b1.Radius,
                'x': round(b1.Center[0], 2),
                'y': round(b1.Center[1], 2),
                'layer': b1.Layer,
                'color': b1.color,
                'linetype': b1.Linetype,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='circles', ext='.csv', folder='csv/'), index=False, sep='$')

    def _read_mtexts(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbMText']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'txt': b1.TextString,
                'height': b1.Height,
                'width': b1.Width,
                'x': round(b1.InsertionPoint[0], 2),
                'y': round(b1.InsertionPoint[1], 2),
                'rot': b1.Rotation,
                'layer': b1.Layer,
                'style': b1.StyleName,
                'color': b1.color,
                'linetype': b1.Linetype,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='mtexts', ext='.csv', folder='csv/'), index=False, sep='$')

    def _read_texts(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbText']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'txt': b1.TextString,
                'height': b1.Height,
                'x': round(b1.InsertionPoint[0], 2),
                'y': round(b1.InsertionPoint[1], 2),
                'rot': b1.Rotation,
                'layer': b1.Layer,
                'style': b1.StyleName,
                'color': b1.color,
                'linetype': b1.Linetype,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='texts', ext='.csv', folder='csv/'), index=False, sep='$')

    def _read_tables(self):
        if self.df_ent_type is None:
            self._read_ent_type()

        df = self.df_ent_type[self.df_ent_type['entity_name']=='AcDbTable']
        if len(df) == 0:
            return None
        ls = []
        for h in df['handle'].values:
            b1 = self.get_obj(h)
            d = {
                'h': h,
                'height': b1.Height,
                'width': b1.Width,
                'rows': b1.Rows,
                'columns': b1.Columns,
                'x': round(b1.InsertionPoint[0], 2),
                'y': round(b1.InsertionPoint[1], 2),
                'layer': b1.Layer,
                'style': b1.StyleName,
                'color': b1.color,
                'linetype': b1.Linetype,
                'linetype_scale': b1.LinetypeScale,
                'lineweight': b1.Lineweight,
                'material': b1.Material,
            }
            ls.append(d)

        df = pd.DataFrame(ls)

        df.to_csv(self._get_path_code(self.doc, prefix='tables', ext='.csv', folder='csv/'), index=False, sep='$')

    def create_df_st_table(self):
        self._create_df_st_table()
        self._create_df_st_v_table()
        self._create_df_st_sh_table()


    def _create_df_st_table(self):
        ls = []
        df = pd.DataFrame(ls, columns=['st', 'pol', 'rd', 'k', 'r', 'c', 'h'])

        for st, name_sx in self.montag_ds_name.items():
            self._set_doc(name_sx)
            for h in self.df_table['h'].tolist():
                b1 = self.get_obj(h)
                xy = b1.InsertionPoint
                df1 = self.find_text(ad.df_text, xy[0], xy[1], -300, 390, -245, 290)
                df1 = self.fdf_match(df1, r'^Полка')
                polka = self.fdf_f_cols(df1, ['txt'])

                if polka is not None:

                    polka = polka.split(' ')[-1]

                    for c in range(b1.Columns):
                        ls1 = []
                        v1 = b1.GetCellValue(0, c)
                        if v1 is None:
                            continue
                        for r in range(2, b1.Rows):
                            v2 = b1.GetCellValue(r, c)
                            if v2 is None:
                                continue
                            ls1.append([st, polka, v1, v2, r, c, h])
                        df = pd.concat([df, pd.DataFrame(ls1, columns=['st', 'pol', 'rd', 'k', 'r', 'c', 'h'])])

        self.df_st_table = df.reset_index(drop=True)
        self.df_st_table = df.astype({'st': 'str', 'pol': 'str', 'rd': 'str', 'k': 'str'})
        self.df_st_table.to_csv(self._get_path_code(prefix='st_table', ext='.csv', folder='csv/'), index=False, sep='$')

    def _create_df_st_v_table(self):

        ls = []
        for st, name_sxema in self.montag_ds_name.items():
            self._set_doc(name_sxema)
            df = ad.fdf_filter(self.df_table, ('rows', 21), ('columns', 3))
            for i in range(len(df)):
                h = df.iloc[i]['h']
                b1 = ad.get_obj(h)
                name = b1.GetCellValue(0, 1)
                ls.append([st, name, h])
        self.df_st_v_table = pd.DataFrame(ls, columns=['st', 'name', 'h'])
        self.df_st_v_table.to_csv(self._get_path_code(prefix='st_v_table', ext='.csv', folder='csv/'), index=False, sep='$')

    def _create_df_st_sh_table(self):

        ls = []
        for st, name_sxema in self.montag_ds_name.items():
            self._set_doc(name_sxema)
            df = ad.fdf_filter(self.df_table, ('rows', 32), ('columns', 2))
            for i in range(len(df)):
                h = df.iloc[i]['h']
                b1 = ad.get_obj(h)
                name = b1.GetCellValue(0, 1) #'2Ш-П'
                pol, pit = name.split('Ш-')
                ls.append([st, pol, pit, name, h])
        self.df_st_sh_table = pd.DataFrame(ls, columns=['st', 'pol', 'pit', 'name', 'h'])
        # self.df_st_sh_table = self.df_st_sh_table.groupby(['st', 'pit'])['pol'].agg(lambda x: list(x)).reset_index()
        self.df_st_sh_table.to_csv(self._get_path_code(prefix='st_sh_table', ext='.csv', folder='csv/'), index=False, sep='$')


    def load_entity_from_csv(self):
        try:
            self.df_poly = self._load_entity_from_csv('polylines')
            self.df_line = self._load_entity_from_csv('lines')
            self.df_circle = self._load_entity_from_csv('circles')
            self.df_mtext = self._load_entity_from_csv('mtexts')
            self.df_text = self._load_entity_from_csv('texts')
            self.df_table = self._load_entity_from_csv('tables')
        except Exception as e:
            logging.error(f'load_entity_from_csv: {e}')

    def load_all_data_from_csv(self):
        dict_dtypes={'st': str, 'pol': str, 'rd': str, 'k': str,}
        self.df_st_table = self._load_data_from_csv('st_table', dict_dtypes=dict_dtypes)
        self.df_st_v_table = self._load_data_from_csv('st_v_table', dict_dtypes=dict_dtypes)
        self.df_st_sh_table = self._load_data_from_csv('st_sh_table', dict_dtypes=dict_dtypes)
        self.df_sx_contacts = self._load_data_from_csv('sx_contacts', dict_dtypes=dict_dtypes)
        self.df_B = self._load_data_from_csv('sx_B', dict_dtypes=dict_dtypes)
        self.df_101 = self._load_data_from_csv('sx_101', dict_dtypes=dict_dtypes)
        self.df_C = self._load_data_from_csv('sx_C', dict_dtypes=dict_dtypes)
        self.df_sx_sh_circle = self._load_data_from_csv('sx_sh_circle', dict_dtypes=dict_dtypes)

    def _load_entity_from_csv(self, prefix):
        path = self._get_path_code(self.doc, prefix=prefix, ext='.csv', folder='csv/')
        if os.path.exists(path):
            df = pd.read_csv(path, sep='$')
            return df
        else:
            return None

    def _load_data_from_csv(self, prefix, dict_dtypes=None):
        path = self._get_path_code(prefix=prefix, ext='.csv', folder='csv/')
        if os.path.exists(path):
            df = pd.read_csv(path, sep='$', dtype=dict_dtypes)
            return df
        else:
            return None

    # def load_polylines(self):
    #     self.df_poly = pd.read_csv(self._get_path_code(prefix='polylines', ext='.csv', folder='csv/'), sep='$')
    #
    # def load_lines(self):
    #     self.df_poly = pd.read_csv(self._get_path_code(prefix='lines', ext='.csv', folder='csv/'), sep='$')
    #
    # def load_mtexts(self):
    #     self.df_poly = pd.read_csv(self._get_path_code(prefix='mtexts', ext='.csv', folder='csv/'), sep='$')
    #
    # def load_tables(self):
    #     self.df_poly = pd.read_csv(self._get_path_code(prefix='tables', ext='.csv', folder='csv/'), sep='$')

    def _get_path_code(self, doc=None, prefix='', suffix='', ext='', folder=''):
        if doc is not None:
            fullname_file = doc.FullName.replace('\\', '$').replace(':', '~')
            path = f'{folder}{prefix}%{fullname_file}%{suffix}{ext}'
        else:
            path = f'{folder}{prefix}{suffix}{ext}'
        return path

    def _read_ent_type(self):
        ls = []
        for ob in self.model:
            ls.append([ob.Handle, ob.EntityName])
        self.df_ent_type = pd.DataFrame(ls, columns=['handle', 'entity_name'])

    def a_point(self, x, y, z=0):
        return win32com.client.VARIANT(pythoncom.VT_ARRAY | pythoncom.VT_R8, (x, y, z))

    def get_obj(self, handle, doc=None):
        if doc is None:
            ob1 = self.model.Database.HandleToObject(handle)
        else:
            ob1 = doc.Database.HandleToObject(handle)
        return ob1


    def get_docs(self):
        self.docs_open = {}
        for d1 in self.app.Documents:
            self.docs_open[d1.FullName] = d1

    def load_docs(self, flag='cm'):
        self.get_docs()

        if flag == 'c' or flag == 'cm':
            for d in self.circuit_ls_name:
                self._load_dwg(d)
        if flag == 'm' or flag == 'cm':
            for d in self.montag_ds_name.values():
                self._load_dwg(d)

        # self.get_docs()
        # time.sleep(1)
        # self._init()

    def _load_dwg(self, dwg_name):
        doc = self.docs_open.get(dwg_name)
        if doc is None:
            doc = self.app.Documents.Open(dwg_name)
            self.docs_open[dwg_name] = doc
        return doc

    def close_docs(self, flag='cm', save=False):
        # self.get_docs()
        if flag == 'c' or flag == 'cm':
            for name_dwg in self.circuit_ls_name:
                self._close_dwg(name_dwg, save=save)
        if flag == 'm' or flag == 'cm':
            for name_dwg in self.montag_ds_name.values():
                self._close_dwg(name_dwg, save=save)
        # self.get_docs()
        # time.sleep(1)
        # self._init()

    def _close_dwg(self, dwg_name, save=False):
        doc = self.docs_open.get(dwg_name)
        if doc is not None:
            doc.Close(save)
            self.docs_open.pop(dwg_name)

    def open_doc(self, path):
        self.doc = self.docs_open.get(path)
        if self.doc is None:
            self.doc = self.app.Documents.Open(path)
            self.docs_open[path] = self.doc
        # self.doc = self.app.Documents.Open(path)
        self.model = self.doc.ModelSpace



    def find_text(self, df_poisk, x, y,  dx_left, dx_right, dy_up, dy_down,  col_name_text='txt',  lst_filter=None, lst_exclude=None, h=None):

        df = df_poisk[(df_poisk['x'] >= x - dx_left) & (df_poisk['x'] <= x + dx_right) & (df_poisk['y'] >= y - dy_down) & (df_poisk['y'] <= y + dy_up)]

        # исключить текущий объект
        if h is not None:
            df = df[df['h'] != h]

        if lst_filter is not None:
            # выбрать только объекты из фильтра
            df = df[df[col_name_text].isin(lst_filter)]

        if lst_exclude is not None:
            # выбрать только объекты из фильтра
            df = df[~df[col_name_text].isin(lst_exclude)]

        return df

    def find_cirle(self, df_poisk, x, y,  dx_left, dx_right, dy_up, dy_down, radius_lt=None, radius_gt=None, h=None):

        df = df_poisk[(df_poisk['x'] >= x - dx_left) & (df_poisk['x'] <= x + dx_right) & (df_poisk['y'] >= y - dy_down) & (df_poisk['y'] <= y + dy_up)]
        # выбрать только объекты с указанным радиусом
        if radius_lt is not None:
            df = df[df['r'] < radius_lt]
        if radius_gt is not None:
            df = df[df['r'] > radius_gt]

        return df

    def find_polyline(self, df_poisk, x, y,  dx_left, dx_right, dy_up, dy_down):
        df = self.df_poly[(self.df_poly['x1'] >= x - dx_left) & (self.df_poly['x1'] <= x + dx_right) & (self.df_poly['y1'] >= y - dy_down) & (self.df_poly['y1'] <= y + dy_up)]

        df1 = self.df_poly[(self.df_poly['x2'] >= x - dx_left) & (self.df_poly['x2'] <= x + dx_right) & (self.df_poly['y2'] >= y - dy_down) & (self.df_poly['y2'] <= y + dy_up)]

        if len(df) != 0:
            return df

        if len(df1) != 0:
            return df1

    # Стативы--------------------------------------------------------------------------------------------------------------
    def st_set_k(self, loc, k, val1=None, val2=None):
        try:
            if type(loc) == str:
                st, p, r = self.get_loc(loc)
            else:
                st, p, r = loc
            df1 = self.fdf_filter(self.df_st_table, ('st', st), ('pol', p), ('rd', r), ('k', k))
            if len(df1) > 0:
                b1 = self.docs_open[self.montag_ds_name[st]]
                b2 = self.get_obj(df1.iloc[0]['h'], b1)
                if val1 is not None:
                    b2.SetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c'] + 1, val1)
                if val2 is not None:
                    b2.SetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c'] + 2, val2)
            else:
                print('Не найден объект в таблице')
        except Exception as e:
            logging.error('Ошибка в st_set_k: {}'.format(e))

    def st_set_v(self, loc, val1=None, val2=None):
        try:
            st=None
            s2 = loc.split('-')
            if len(s2) == 3:
                st, name, k = s2
                k_row_col = self.ds_st_v_row_col[k]
            elif len(s2) == 2:
                name, k = s2
            else:
                logging.error('Не верный формат строки')

            k_row_col = self.ds_st_v_row_col[k]
            df1 = self.fdf_filter(self.df_st_v_table, ('st', st), ('name', name))
            if len(df1) > 0:
                b1 = self.docs_open[self.montag_ds_name[st]]
                b2 = self.get_obj(df1.iloc[0]['h'], b1)
                if val1 is not None:
                    b2.SetCellValue(k_row_col[0], k_row_col[1], val1)
                if val2 is not None:
                    b2.SetCellValue(k_row_col[0] + 1, k_row_col[1], val2)
            else:
                logging.warning('Не найден объект в таблице')
        except Exception as e:
            logging.error('Ошибка в st_set_v: {}'.format(e))

    def st_set_sh(self, st, pol, k, val1=None):
        try:
            df1 = self.fdf_filter(self.df_st_sh_table, ('st', st), ('pol', pol))
            if len(df1) > 0:
                b1 = self.docs_open[self.montag_ds_name[st]]
                b2 = self.get_obj(df1.iloc[0]['h'], b1)
                b2.SetCellValue(int(k), 0, val1)
            else:
                logging.warning('Не найден объект в таблице')
        except Exception as e:
            logging.error('Ошибка в st_set_v: {}'.format(e))


    def txtdraw_replace(self, h, val, doc=None):
        if doc is None:
            doc = self.doc

        b1 = self.get_obj(h, doc)
        b1.TextString = val

    def st_init_v(self):
        ls_B = ['В15', 'В25', 'В16', 'В26', 'В17', 'В27', 'В18', 'В28', 'В19', 'В29', 'В110', 'В111', 'В112', 'В210',
                'В211', 'В212', 'Б9', 'Б10', 'Б11', 'Б12', 'Б4', 'Б3', 'Б2', 'Б1']
        ls_st = list(self.montag_ds_name.keys())
        ls = []
        for st in ls_st:
            for b in ls_B:
                for i in range(1, 21):
                    ls.append([st, b, i])
        self.df_st_b = pd.DataFrame(ls, columns=['st', 'b', 'n'])
        self.df_st_b[['l1', 'l2', 'p']] = ''



    def st_montag_B(self):
        for name_sxema in self.circuit_ls_name:
            try:
                self._set_doc(name_sxema)
                self._st_montag_B(name_sxema)
            except Exception as e:
                logging.error(f'Ошибка st_montag_B в схеме {name_sxema} {e}')

    def _st_montag_B(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)

        df_B = self.fdf_filter(self.df_B, ('name_sxema', name_sxema))

        for i in range(len(df_B)):
            try:
                res = df_B.iloc[i]
                text_b = res['txt']
                df1, df_name1 = self._find_link_by_b(res['x'], res['y'], 10, 0, 5, 5, side='left', name_sxema=name_sxema)
                df2, df_name2 = self._find_link_by_b(res['x'], res['y'], 0, 0, 5, 5, side='right', name_sxema=name_sxema)
                loc = None
                contact = None
                df_name_st = None
                df_name_no_st = None
                if df_name1 == 'df_contacts':
                    loc = df1.iloc[0]['loc']
                    df_cont = df1
                    contact = df1.iloc[0]['k']
                    b1_dev = df1.iloc[0]['b1']
                    b2_dev = df1.iloc[0]['b2']
                    df_name_st = df_name1
                if df_name1 == 'df_101' or df_name1 == 'df_C':
                    df_no_cont = df1
                    b_st = df1.iloc[0]['txt']
                    df_name_no_st = df_name1

                if df_name2 == 'df_contacts':
                    loc = df2.iloc[0]['loc']
                    df_cont = df2
                    contact = df2.iloc[0]['k']
                    b1_dev = df2.iloc[0]['b1']
                    b2_dev = df2.iloc[0]['b2']
                    df_name_st = df_name2
                if df_name2 == 'df_101' or df_name2 == 'df_C':
                    df_no_cont = df2
                    b_st = df2.iloc[0]['txt']
                    df_name_no_st = df_name2

                if loc is not None and df_name_no_st is not None:
                    st, p, r = self.get_loc(loc)
                    ls = self.ds_st_b[df_name_no_st]
                    if 'Б' in text_b:
                        ls = self.ds_st_b['df_bok']
                    df_t = self.df_st_b[
                        (self.df_st_b['st'] == st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1'] == '')]
                    df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                    df_bb = df_t.head(1)
                    b, n = df_bb['b'].values[0], df_bb['n'].values[0]  #'B110', 1
                    s_b = f'{st}-{b}-{n}'

                    self.txtdraw_replace(res['h'], s_b)
                    self.st_set_k(loc, contact, f'{b}-{n}')
                    self.st_set_v(f'{st}-{b}-{n}', val1=f'{p}{r}-{contact}', val2=b_st)
                    # self.add_st_v(st, b, str(n), [f'{p}{r}-{contact}', b_st])

                    # self.set_val_df(self.df_st_b, df_bb['ref'], 'l1', loc)
                    # self.set_val_df(self.df_st_b, df_bb['ref'], 'l2', b_st)
                    self.df_st_b.loc[df_bb.index, 'l1'] =  loc
                    self.df_st_b.loc[df_bb.index, 'l2'] =  b_st

                    if self.fdf_f_cols(df_cont, ['b1']) is None:
                        # self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b1', f'{b}-{n}')
                        self.df_sx_contacts.loc[df_cont.index, 'b1'] =  f'{b}-{n}'
                    else:
                        # self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b2', f'{b}-{n}')
                        self.df_sx_contacts.loc[df_cont.index, 'b2'] =  f'{b}-{n}'

                if loc is None and df_name_no_st is not None:
                    st = res['txt'][0]
                    ls = self.ds_st_b[df_name_no_st]
                    if 'Б' in text_b:
                        ls = self.ds_st_b['df_bok']
                    df_t = self.df_st_b[
                        (self.df_st_b['st'] == st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1'] == '')]
                    df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                    df_bb = df_t.head(1)
                    b, n = df_bb['b'].values[0], df_bb['n'].values[0]  #'B110', 1
                    s_b = f'{st}-{b}-{n}'

                    self.txtdraw_replace(res['h'], s_b)
                    self.st_set_v(f'{st}-{b}-{n}', val2=b_st)
                    # self.add_st_v(st, b, str(n), ['', b_st])

                    # self.set_val_df(self.df_st_b, df_bb['ref'], 'l1', 'резерв')
                    # self.set_val_df(self.df_st_b, df_bb['ref'], 'l2', b_st)
                    self.df_st_b.loc[df_bb.index, 'l1'] =  'резерв'
                    self.df_st_b.loc[df_bb.index, 'l2'] =  b_st
            except Exception as e:
                logging.error(f'Ошибка в {text_b} $ {loc} {contact} {e}')



    def _find_link_by_b(self, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left', name_sxema=None):
        if name_sxema is None:
            name_sxema = self.doc.Name
        df_name = None
        df = None
        for i in range(30):
            df_sx_contacts = self.fdf_filter(self.df_sx_contacts, ('name_sxema', name_sxema))
            df = self.find_text(df_sx_contacts,x, y, dx_left, dx_right, dy_top, dy_bottom)
            if len(df) > 0:
                df_name = 'df_contacts'
                break

            df_101 = self.fdf_filter(self.df_101, ('name_sxema', name_sxema))
            df = self.find_text(df_101, x, y, dx_left, dx_right, dy_top, dy_bottom)
            if len(df) > 0:
                df_name = 'df_101'
                break

            df_C = self.fdf_filter(self.df_C, ('name_sxema', name_sxema))
            df = self.find_text(df_C, x, y, dx_left, dx_right, dy_top, dy_bottom)
            if len(df) > 0:
                df_name = 'df_C'
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if df_name is not None:
            return df, df_name
        return None, None

    def _find_link_by_sh(self, df_poisk, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left'):
        df = None
        for i in range(30):
            df = self.find_cirle(df_poisk, x, y, dx_left, dx_right, dy_top, dy_bottom)
            if len(df) > 0:
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if len(df) > 0:
            return df
        return None



    def get_xy(self, df, h):
        df = df[df['h']==h]
        if len(df) == 0:
            return None
        x = df.iloc[0]['x']
        y = df.iloc[0]['y']
        return x, y

    def get_h(self, df, h):
        df = df[df['h']==h]
        return df

    def fdf_match(self, df, match, col_name_text='txt'):
        return df[df[col_name_text].str.match(match)]

    def fdf_h(self, df, h):
        df = df[df['h']==h]
        return df

    def fdf_h_cols(self, df, h, cols):
        df = df[df['h']==h][cols]
        if len(cols) == 1:
            return df.iloc[0][cols[0]]
        if len(df) == 0:
            return None
        return df.iloc[0].tolist()

    def fdf_f(self, df):
        if df is None:
            return None
        if len(df) == 0:
            return None
        return df.iloc[0]

    def fdf_f_cols(self, df, cols):
        if df is None:
            return None
        if len(df) == 0:
            return None
        if len(cols) == 1:
            return df.iloc[0][cols[0]]
        return df.iloc[0][cols].tolist()

    def fdf_f_h(self, df):
        if df is None:
            return None
        if len(df) == 0:
            return None
        return df.iloc[0]['h']

    def fdf_filter(self, df, col1_val1, col2_val2=None, col3_val3=None, col4_val4=None, col5_val5=None, col6_val6=None, col7_val7=None, col8_val8=None, col9_val9=None, col10_val10=None):
            col1, val1 = col1_val1
            df1 = df[df[col1] == val1]
            if col2_val2 is not None:
                col2, val2 = col2_val2
                df1 = df1[df1[col2] == val2]

            if col3_val3 is not None:
                col3, val3 = col3_val3
                df1 = df1[df1[col3] == val3]

            if col4_val4 is not None:
                col4, val4 = col4_val4
                df1 = df1[df1[col4] == val4]

            if col5_val5 is not None:
                col5, val5 = col5_val5
                df1 = df1[df1[col5] == val5]

            if col6_val6 is not None:
                col6, val6 = col6_val6
                df1 = df1[df1[col6] == val6]

            if col7_val7 is not None:
                col7, val7 = col7_val7
                df1 = df1[df1[col7] == val7]

            if col8_val8 is not None:
                col8, val8 = col8_val8
                df1 = df1[df1[col8] == val8]

            if col9_val9 is not None:
                col9, val9 = col9_val9
                df1 = df1[df1[col9] == val9]

            if col10_val10 is not None:
                col10, val10 = col10_val10
                df1 = df1[df1[col10] == val10]

            return df1

    def fdf_set_val(self, df, h, col, value, name_sxema=None):
        if df is not None:
            if name_sxema is None:
                ser = df[df['h'] == h][col]
                if len(ser) > 0:
                    df.loc[df['h'] == h, col] = value
            else:
                ser = df[(df['h'] == h) & (df['name_sxema'] == name_sxema)][col]
                if len(ser) > 0:
                    df.loc[(df['h'] == h) & (df['name_sxema'] == name_sxema), col] = value
        return df

    def _direction_contact(self, df):
            if len(df) > 1:
                c1 = df.iloc[0]['txt']
                c2 = df.iloc[1]['txt']
                x1 = df.iloc[0]['x']
                x2 = df.iloc[1]['x']
                y1 = df.iloc[0]['y']
                y2 = df.iloc[1]['y']

                dx = abs(x1 - x2)
                dy = abs(y1 - y2)

                if dx > 2:
                    if x1 < x2:
                        return {c1: ['horizontal', 'left'], c2: ['horizontal', 'right']}
                    else:
                        return {c1: ['horizontal', 'right'], c2: ['horizontal', 'left']}
                if dy > 2:
                    if y1 > y2:
                        return {c1: ['vertical', 'top'], c2: ['vertical', 'bottom']}
                    else:
                        return {c1: ['vertical', 'bottom'], c2: ['vertical', 'top']}

    def get_loc(self, loc):
            ls = loc.split('-')
            st = ls[0]
            pr = ls[1]
            if len(pr) == 2:
                p = pr[0]
                r = pr[1]
            else:
                p = pr[0:2]
                r = pr[2]
            return st, p, r


    def _rep_xy_on_circle(self, row):
        df = self.find_cirle(self.df_circle, row['x'], row['y'], 10, 10, 0, 8, radius_lt=2)
        if len(df) > 0:
            row['x'] = df.iloc[0]['x']
            row['y'] = df.iloc[0]['y']
        return row
    
    def sx_search_contacts_and_v_all(self):
        self.df_sx_contacts = None
        self.df_B = None
        self.df_C = None
        self.df_101 = None

        for name_sxema in self.circuit_ls_name:
            try:
                self._set_doc(name_sxema)
                self._sx_search_contacts(name_sxema)
                self._sx_search_v(name_sxema)
            except Exception as e:
                logging.error(f'Ошибка sx_search_contacts_and_v в схеме {name_sxema} {e}')
    
    def _sx_search_contacts(self, name_sxema, apart=False):

        if apart:
            self._set_doc(name_sxema)

        df_mtext = self.df_mtext
        df_dev = self.fdf_match(df_mtext, r'^[1-3]-\d{2,3}')
        lsh = df_dev['h'].tolist()

        ls = []

        for h in lsh:
            try:
                loc = self.fdf_h_cols(df_mtext, h, ['txt'])
                x, y = self.fdf_h_cols(df_mtext, h, ['x', 'y'])

                df_name = self.find_text(df_mtext, x, y, 12, 12, 0, 5, lst_exclude=self.ls_contact, h=h)
                if len(df_name) == 0:
                    logging.error(f'Не найдено имя устройства {loc}')
                    df_name = self.find_text(df_mtext, x, y, 20, 20, 0, 20, lst_exclude=self.ls_contact, h=h)
                h_name = self.fdf_f_h(df_name)
                t_name = self.fdf_f_cols(df_name, ['txt'])
                # x, y = self.fdf_f_cols(df_name, ['x', 'y'])
                df_circle = self.find_cirle(self.df_circle, x, y, 20, 20, 8, 9, radius_gt=3)
                if len(df_circle) > 0:
                    h_c= self.fdf_f_h(df_circle)
                    x, y = self.fdf_f_cols(df_circle, ['x', 'y'])
                    df_k = self.find_text(df_mtext, x, y, 10, 10, 10, 10, lst_filter=self.ls_contact, h=h)
                else:
                    df_k = self.find_text(df_mtext, x, y, 20, 20, 0, 12, lst_filter=self.ls_contact, h=h)

                if len(df_k) == 0:
                    logging.error(f'Не найден номер контакта {loc}')
                    continue

                if df_k['txt'].tolist() == ['1', '2', '3', '4']:
                    df_k = df_k[df_k['txt'].isin(['2', '4'])]
                ds = self._direction_contact(df_k)
                group = loc + ';' + ':'.join(df_k['txt'].values.tolist())

                for j in range(len(df_k)):
                    k = df_k.iloc[j]['txt']
                    t_ref = df_k.iloc[j]['h']
                    t_x = df_k.iloc[j]['x']
                    t_y = df_k.iloc[j]['y']
                    st, p, r = self.get_loc(loc)
                    ls.append([loc, st, p, r, t_name, k, t_ref, t_x, t_y, group, ds[k][0],
                                               ds[k][1]])
            except Exception as e:
                logging.error(f'Ошибка _sx_search_contacts - {loc} контакт: {k} - {df_k["txt"].tolist()} - {e}')


        df = pd.DataFrame(ls, columns=['loc', 'st', 'pol', 'rd', 'name', 'k', 'h', 'x', 'y', 'group', 'direction',
                                                'side'])

        df['name_sxema'] = name_sxema
        df[['b1', 'b2', 'l1', 'l2', 'pit', 'pit_h_circle']] = None
        # self.df_sx_contacts = df_sx_contacts.reset_index(drop=True)
        df =  df.astype({'st': 'str', 'pol': 'str', 'rd': 'str', 'k': 'str'})

        if self.df_sx_contacts is None:
            self.df_sx_contacts = df
        else:
            self.df_sx_contacts = pd.concat([self.df_sx_contacts, df], axis=0)
            self.df_sx_contacts = self.df_sx_contacts.reset_index(drop=True)

        self.save_sx_contacts()

    def save_sx_contacts(self):
        self.df_sx_contacts.to_csv(self._get_path_code(prefix='sx_contacts', ext='.csv', folder='csv/'), index=False, sep='$')

        
    def _sx_search_v(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)

        df_B = self.fdf_match(self.df_mtext, r'^[1-3]-[ВБ]')
        df_B = df_B.apply(self._rep_xy_on_circle, axis=1)
        df_B = df_B.reset_index(drop=True)
        df_B['name_sxema'] = name_sxema
        # self.df_B[['x', 'y']] = self.df_B[['x', 'y']].apply(lambda x: x.round(2))

        df_101 = self.fdf_match(self.df_mtext, r'^10[1-5]-')
        df_101 = df_101.apply(self._rep_xy_on_circle, axis=1)
        df_101 = df_101.reset_index(drop=True)
        df_101['name_sxema'] = name_sxema
        # self.df_101[['x', 'y']] = self.df_101[['x', 'y']].apply(lambda x: x.round(2))

        df_C = self.fdf_match(self.df_mtext, r'^C[1-8]:')
        df_C = df_C.apply(self._rep_xy_on_circle, axis=1)
        df_C = df_C.reset_index(drop=True)
        df_C['name_sxema'] = name_sxema
        # self.df_C[['x', 'y']] = self.df_C[['x', 'y']].apply(lambda x: x.round(2))

        # отсортировать по x возрастанию потом y по убыванию
        df_B = df_B.sort_values(by=['x', 'y'], ascending=[True, False])

        if self.df_B is None:
            self.df_B = df_B
        else:
            self.df_B = pd.concat([self.df_B, df_B], axis=0)
            self.df_B = self.df_B.reset_index(drop=True)
        self.df_B.to_csv(self._get_path_code(prefix='sx_B', ext='.csv', folder='csv/'), index=False, sep='$')

        if self.df_101 is None:
            self.df_101 = df_101
        else:
            self.df_101 = pd.concat([self.df_101, df_101], axis=0)
            self.df_101 = self.df_101.reset_index(drop=True)
        self.df_101.to_csv(self._get_path_code(prefix='sx_101', ext='.csv', folder='csv/'), index=False, sep='$')

        if self.df_C is None:
            self.df_C = df_C
        else:
            self.df_C = pd.concat([self.df_C, df_C], axis=0)
            self.df_C = self.df_C.reset_index(drop=True)
        self.df_C.to_csv(self._get_path_code(prefix='sx_C', ext='.csv', folder='csv/'), index=False, sep='$')


    # Питание---------------------------------------------------------------------------------
    def sx_search_sh_circle(self):

        self.df_sx_sh_circle = None

        for name_sxema in self.circuit_ls_name:
            try:
                self._set_doc(name_sxema)
                self._sx_search_sh_circle(name_sxema)
                self._sx_set_pit_contact(name_sxema)
                self.save_sx_contacts()
            except Exception as e:
                logging.error(f'Ошибка sx_search_sh_circle в схеме {name_sxema} {e}')

        # for name_sxema in self.circuit_ls_name:
        #     try:
        #         self._set_doc(name_sxema)
        #         self._sx_set_pit_contact(name_sxema)
        #         self.save_sx_contacts()
        #     except Exception as e:
        #         logging.error(f'Ошибка sx_search_sh_circle в схеме {name_sxema} {e}')

    def _sx_search_sh_circle(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)

        df_sh = self.fdf_match(self.df_mtext, r'[1-3]-Ш')
        lsh = df_sh['h'].tolist()

        for i in range(len(df_sh)):
            h = df_sh.iloc[i]['h']
            x, y = self.fdf_h_cols(self.df_mtext, h, ['x', 'y'])
            ds = self.find_text(self.df_mtext, x, y, 8, 8, 0, 6, lst_filter=self.ls_st_sh_name, h=h)
            if len(ds) > 0:
                df2 = None
                name_sh = self.fdf_f_cols(ds, ['txt'])  # 'П'
                x, y = self.fdf_f_cols(ds, ['x', 'y'])
                ds1 = self.find_cirle(self.df_circle, x, y, 5, 5, 0, 10, radius_lt=2)
                if len(ds1) > 0:
                    df2 = ds1
                    df2['pit'] = name_sh
                    df2['h_name_sh'] = h
                    df2['one_k'] = 'one'
                    df2['h_k'] = None
                    df2['name_sxema'] = name_sxema

                df_lines = self.find_polyline(self.df_poly, x, y, 5, 5, 0, 10)

                if len(df_lines) > 0:
                    is_line = False
                    for i in range(len(df_lines)):
                        res = df_lines.iloc[i]
                        if res['x1'] == res['x2']:
                            is_line = True
                            break
                    if is_line:
                        df2 = self.find_cirle(self.df_circle, x, y, 5, 5, 0, 180, radius_lt=2)
                        df2['pit'] = name_sh
                        df2['h_name_sh'] = h
                        df2['one_k'] = 'many'
                        df2['h_k'] = None
                        df2['name_sxema'] = name_sxema
                        ls_filtr = ['**', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30' ]
                        for i in range(len(df2)):
                            x, y = df2.iloc[i][['x', 'y']]
                            h1 = df2.iloc[i]['h']
                            df3 = ad.find_text(ad.df_mtext, x, y, 6, 6, 3, 3, lst_filter=ls_filtr, h=h)
                            h_k = ad.fdf_f_cols(df3, ['h'])
                            ad.fdf_set_val(df2, h1, 'h_k', h_k, name_sxema)

                if df2 is not None:

                    if self.df_sx_sh_circle is None:
                        self.df_sx_sh_circle = df2.copy()
                    else:
                        self.df_sx_sh_circle = self.df_sx_sh_circle.append(df2)
                        self.df_sx_sh_circle = self.df_sx_sh_circle.reset_index(drop=True)

                    self.df_sx_sh_circle.to_csv(self._get_path_code(prefix='sx_sh_circle', ext='.csv', folder='csv/'), index=False, sep='$')


    def _sx_set_pit_contact(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)
        df1 = self.df_sx_contacts[self.df_sx_contacts['b2'].isnull()]
        df1 = ad.fdf_filter(df1, ('name_sxema', name_sxema))
        for i in range(len(df1)):
            res = df1.iloc[i]
            h = res['h']
            df_sx_sh_circle = self.fdf_filter(self.df_sx_sh_circle, ('name_sxema', name_sxema))
            df2 = self._find_link_by_sh(df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
            if df2 is not None:
                pit = df2['pit'].values[0]
                h_circle = df2['h'].values[0]

                self.df_sx_contacts = self.fdf_set_val(self.df_sx_contacts, h, 'pit', pit, name_sxema)
                self.df_sx_contacts = self.fdf_set_val(self.df_sx_contacts, h, 'pit_h_circle', h_circle, name_sxema)


    def st_montag_shina(self):
        for st_name in self.ls_st_name:
            for sh_name in self.ls_st_sh_name:
                try:
                    df1 = self.fdf_filter(self.df_sx_contacts, ('st', st_name), ('pit', sh_name))
                    if len(df1) > 0:
                        df2 = df1.copy()
                        df2 = df2.astype({'pol': 'int', 'rd': 'int', 'k': 'int'})
                        df2 = df2.sort_values(by=['pol', 'rd', 'k'])
                        df2 = df2.astype({'pol': 'str', 'rd': 'str', 'k': 'str'})
                        for i in range(len(df2)):
                            loc = df2['loc'].values[i]
                            k_dev = df2['k'].values[i]
                            h_pit_circle = df2['pit_h_circle'].values[i]
                            name_sxema = df2['name_sxema'].values[i]
                            polka_shina = str(self.get_nearest_sh(loc, sh_name))
                            k_shina = self._st_get_contact_sh(st_name,  polka_shina, sh_name, loc)

                            p = df2['pol'].values[i]
                            r = df2['rd'].values[i]
                            if p == polka_shina:
                                loc_to = f'Ш-{k_shina}'
                                loc_to_sh = f'{r}-{k_dev}'
                            else:
                                loc_to = f'{polka_shina}Ш-{k_shina}'
                                loc_to_sh = f'{p}{r}-{k_dev}'

                            self.st_set_k(loc, k_dev, loc_to, sh_name)
                            self.st_set_sh(st_name, polka_shina, k_shina, loc_to_sh)

                            df3 = self.fdf_filter(self.df_sx_sh_circle, ('h', h_pit_circle), ('name_sxema', name_sxema))
                            h_name_sh, one_k, h_k =df3['h_name_sh'].values[0], df3['one_k'].values[0], df3['h_k'].values[0]
                            doc = self.docs_open[name_sxema]
                            if one_k == 'one':
                                self.txtdraw_replace(h_name_sh, f'{st_name}-Ш{polka_shina}-{k_shina}', doc)
                            else:
                                self.txtdraw_replace(h_name_sh, f'{st_name}-Ш{polka_shina}', doc)
                                self.txtdraw_replace(h_k, k_shina, doc)

                except Exception as e:
                    logging.error(f'Ошибка st_montag_shina {st_name} {sh_name} {loc} {k_dev} {polka_shina} {k_shina} {e}')


    def _st_get_contact_sh(self, st,  polka, sh, loc):
        df1 = self.fdf_filter(self.df_st_shina_montag, ('st', st), ('sh', sh), ('polka', polka))
        if len(df1) == 0:
            self.df_st_shina_montag = self.df_st_shina_montag.append({'st': st, 'sh': sh, 'loc': loc, 'k': '1', 'polka': polka}, ignore_index=True)
            return '1'
        else:
            cont = str(int(df1.tail(1)['k'].values[0]) + 1)
            self.df_st_shina_montag = self.df_st_shina_montag.append({'st': st, 'sh': sh, 'loc': loc, 'k': cont, 'polka': polka}, ignore_index=True)
            return cont

    def get_nearest_sh(self, loc, pit):
        st, p, r = self.get_loc(loc)
        df = self.fdf_filter(self.df_st_sh_table, ('st', st), ('pit', pit))
        if len(df) > 0:
            ls_pol = df['pol'].to_list()
            ls_pol = [int(x) for x in ls_pol]
            return self._find_nearest(ls_pol, int(p))

    # выбрать из списка ls1 число ближйшее к заданному
    def _find_nearest(self, ls, value):
        ls = np.array(ls)
        idx = (np.abs(ls - value)).argmin()
        return ls[idx]

    # def add_dwg_circuit(self, dwg_path):
    #     app1 = QtWidgets.QApplication([])
    #     self.docs_circuit = QtWidgets.QFileDialog.getOpenFileNames(None, 'Выберите принципиальные схемы', r'd:\Фабричная\Фабричная', 'dwg files (*.dwg)')[0]
    #
    # def add_dwg_montag(self, dwg_path):
    #     app1 = QtWidgets.QApplication([])
    #     self.docs_montag = QtWidgets.QFileDialog.getOpenFileNames(None, 'Выберите монтажные схемы', r'd:\Фабричная\Фабричная', 'dwg files (*.dwg)')[0]


In [4]:
ad = Acad()

In [5]:
ad.load_docs('cm')

In [74]:
ad.close_docs('cm')

In [6]:
# ad.read_entity_to_csv_all_docs()

In [8]:
ad._init()
# ad.get_docs()
ad.load_entity_from_csv()
ad.load_all_data_from_csv()
ad.st_init_v()

In [6]:
# ad.sx_search_contacts_and_v_all()

In [7]:
# ad.create_df_st_table()

In [22]:
ad.sx_search_sh_circle()

In [9]:
ad.st_montag_B()
ad.sx_search_sh_circle()
ad.st_montag_shina()

In [11]:
ad.fdf_filter(ad.df_sx_sh_circle, ('h', '29B7E'))

,h,r,x,y,layer,color,linetype,linetype_scale,lineweight,material,pit,h_name_sh,one_k,h_k,name_sxema
159,29B7E,0.750078,1648.06,128.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,М,29FD8,one,None,D:\Фабричная\Фабричная\Принципиальные схемы\25...


In [23]:
ad.df_sx_contacts['h'].value_counts()


2E8C1    2
2E8EF    2
2E8F3    2
2E8BF    2
2E8D5    2
        ..
336AB    1
336AE    1
33976    1
33979    1
2EAA5    1
Name: h, Length: 929, dtype: int64

In [15]:
name_sxema =ad.circuit_ls_name[0]
name_sxema

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg'

In [16]:
res = df1
h = res['h']
df_sx_sh_circle = ad.fdf_filter(ad.df_sx_sh_circle, ('name_sxema', name_sxema))
df_sx_sh_circle

,h,r,x,y,layer,color,linetype,linetype_scale,lineweight,material,pit,h_name_sh,one_k,h_k,name_sxema
0,2D056,0.749800,7944.06,130.73,PDF15_Geometry,256,ByLayer,1.0,25,ByLayer,П,2D0B1,many,2D0B3,D:\Фабричная\Фабричная\Принципиальные схемы\25...
1,2D058,0.750025,7944.06,120.73,PDF15_Geometry,256,ByLayer,1.0,25,ByLayer,П,2D0B1,many,2D0B4,D:\Фабричная\Фабричная\Принципиальные схемы\25...
2,2D05B,0.750025,7944.06,110.73,PDF15_Geometry,256,ByLayer,1.0,25,ByLayer,П,2D0B1,many,2D0B5,D:\Фабричная\Фабричная\Принципиальные схемы\25...
3,2D05E,0.750025,7944.06,100.73,PDF15_Geometry,256,ByLayer,1.0,25,ByLayer,П,2D0B1,many,2D0B6,D:\Фабричная\Фабричная\Принципиальные схемы\25...
4,2D061,0.750025,7944.06,90.73,PDF15_Geometry,256,ByLayer,1.0,25,ByLayer,П,2D0B1,many,2D0B7,D:\Фабричная\Фабричная\Принципиальные схемы\25...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,36F76,0.750025,8784.06,60.73,PDF16_Geometry,256,ByLayer,1.0,25,ByLayer,П,2DD59,many,36F8B,D:\Фабричная\Фабричная\Принципиальные схемы\25...
72,36F78,0.750025,8784.06,50.73,PDF16_Geometry,256,ByLayer,1.0,25,ByLayer,П,2DD59,many,36F8C,D:\Фабричная\Фабричная\Принципиальные схемы\25...
73,36F7A,0.749800,8784.06,40.73,PDF16_Geometry,256,ByLayer,1.0,25,ByLayer,П,2DD59,many,36F8D,D:\Фабричная\Фабричная\Принципиальные схемы\25...
74,36F7C,0.750025,8784.06,30.73,PDF16_Geometry,256,ByLayer,1.0,25,ByLayer,П,2DD59,many,36F8E,D:\Фабричная\Фабричная\Принципиальные схемы\25...


In [17]:
df2 = ad._find_link_by_sh(df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
df2

In [18]:
if df2 is not None:
    print('dgdf')

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg'

In [8]:
ad.circuit_ls_name

['D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.4 Схемы РЦ Чет..dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.9 Переезды.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.10 Схемы увязок.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.11 Ограждение.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.12 Загальні схеми.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.13 Контр.предохр..dwg']

In [9]:
nsx = ad.circuit_ls_name[3]
nsx

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.10 Схемы увязок.dwg'

In [12]:
df = ad.fdf_filter(ad.df_sx_sh_circle, ('name_sxema', nsx))
df

,h,r,x,y,layer,color,linetype,linetype_scale,lineweight,material,pit,name_sxema
196,2E3C6,0.750078,1438.06,128.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,М,D:\Фабричная\Фабричная\Принципиальные схемы\25...
197,2E3C3,0.749852,1438.06,144.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
198,31063,0.749852,1423.06,143.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
199,31065,0.749852,1423.06,127.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
200,31068,0.749852,1423.06,111.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
201,310DF,0.750078,1438.06,88.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,М,D:\Фабричная\Фабричная\Принципиальные схемы\25...
202,310DE,0.749852,1438.06,104.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
203,310E5,0.750078,1438.06,48.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,М,D:\Фабричная\Фабричная\Принципиальные схемы\25...
204,310E4,0.749852,1438.06,64.73,PDF16_Geometry,1,ByLayer,1.0,25,ByLayer,П,D:\Фабричная\Фабричная\Принципиальные схемы\25...
205,310FD,0.750078,1443.06,4.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,М,D:\Фабричная\Фабричная\Принципиальные схемы\25...


In [14]:
df1 = ad.fdf_f_cols(df, ['x', 'y', 'pit'])
df1

[1438.06, 128.73, 'М']

In [15]:
df_sh = ad.fdf_match(ad.df_mtext, r'[1-3]-Ш')
df_sh

,h,txt,height,width,x,y,rot,layer,style,color,linetype,linetype_scale,lineweight,material
405,2E768,1-Ш,2.222633,0.0,1432.07,134.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
411,2E770,1-Ш,2.222633,0.0,1432.07,150.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
620,3106D,1-Ш,2.222633,0.0,1418.49,149.45,0.0,PDF2_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
635,310E0,1-Ш,2.222633,0.0,1432.07,94.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
637,310E2,1-Ш,2.222633,0.0,1432.07,110.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
639,310E6,1-Ш,2.222633,0.0,1432.07,54.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
641,310E8,1-Ш,2.222633,0.0,1432.07,70.45,0.0,PDF16_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer
645,31107,1-Ш,2.222633,0.0,1439.33,10.45,0.0,PDF2_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer


In [31]:
i=2
h = df_sh.iloc[i]['h']
x, y = ad.fdf_h_cols(ad.df_mtext, h, ['x', 'y'])
ds = ad.find_text(ad.df_mtext, x, y, 8, 8, 0, 6, lst_filter=ad.ls_st_sh_name, h=h)
# ds
name_sh = ad.fdf_f_cols(ds, ['txt'])  # 'П'
x, y = ad.fdf_f_cols(ds, ['x', 'y'])
ds1 = ad.find_cirle(ad.df_circle, x, y, 5, 5, 0, 10, radius_lt=2)
# ds1
x, y = ad.fdf_f_cols(ds1, ['x', 'y'])
df_lines = ad.find_polyline(ad.df_poly, x, y, 5, 5, 0, 10)
df_lines

,h,length,x1,y1,x2,y2,points,layer,color,linetype,closed,area,linetype_scale,lineweight,material,thickness
435,31064,1.249894,1421.06,143.73,1422.31,143.73,"(1421.0554460433523, 143.72991277718575, 1422....",PDF2_Geometry,1,ByLayer,False,0.0,1.0,25,ByLayer,0.0
437,31067,14.499901,1423.06,142.98,1423.06,128.48,"(1423.055347202382, 142.979905745241, 1423.055...",PDF2_Geometry,256,ByLayer,False,0.0,1.0,25,ByLayer,0.0


In [34]:
df2 = ad.find_cirle(ad.df_circle, x, y, 5, 5, 0, 180, radius_lt=2)
df2['pit'] = name_sh
df2['h_name_sh'] = h
df2['one_k'] = 'many'
df2['h_k'] = None
df2

,h,r,x,y,layer,color,linetype,linetype_scale,lineweight,material,pit,h_name_sh,one_k,h_k
164,31063,0.749852,1423.06,143.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,None
165,31065,0.749852,1423.06,127.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,None
166,31068,0.749852,1423.06,111.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,None


In [36]:
ls_filtr = ['**', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30' ]
for i in range(len(df2)):
    x, y = df2.iloc[i][['x', 'y']]
    h1 = df2.iloc[i]['h']
    df3 = ad.find_text(ad.df_mtext, x, y, 3, 3, 3, 3, lst_filter=ls_filtr, h=h)
    h_k = ad.fdf_f_cols(df3, ['h'])
    ad.fdf_set_val(df2, h1, 'h_k', h_k)
df2
df3

,h,r,x,y,layer,color,linetype,linetype_scale,lineweight,material,pit,h_name_sh,one_k,h_k
164,31063,0.749852,1423.06,143.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,3106F
165,31065,0.749852,1423.06,127.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,3106C
166,31068,0.749852,1423.06,111.73,PDF2_Geometry,1,ByLayer,1.0,25,ByLayer,П,3106D,many,3106B


In [32]:
ad.find_text(ad.df_mtext, x, y, 3, 3, 3, 3, lst_filter=ls_filtr, h=h)

,h,txt,height,width,x,y,rot,layer,style,color,linetype,linetype_scale,lineweight,material
622,3106F,**,1.778111,0.0,1420.7,144.24,0.0,PDF2_Text,ISOCPEUR,1,ByLayer,1.0,-1,ByLayer


In [28]:
df_lines = ad.find_polyline(ad.df_poly, x, y, 5, 5, 0, 10)
df_lines

,h,length,x1,y1,x2,y2,points,layer,color,linetype,closed,area,linetype_scale,lineweight,material,thickness
46,2E267,41.250034,1438.81,104.73,1477.06,101.73,"(1438.805494940243, 104.72989984869241, 1442.0...",PDF16_Geometry,3,ByLayer,False,35.218678,1.0,25,ByLayer,0.0


In [ ]:
def _sx_search_sh_circle(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)

        df_sh = self.fdf_match(self.df_mtext, r'[1-3]-Ш')
        lsh = df_sh['h'].tolist()

        for i in range(len(df_sh)):
            h = df_sh.iloc[i]['h']
            x, y = self.fdf_h_cols(self.df_mtext, h, ['x', 'y'])
            ds = self.find_text(self.df_mtext, x, y, 8, 8, 0, 6, lst_filter=self.ls_st_sh_name, h=h)
            if len(ds) > 0:
                df2 = None
                name_sh = self.fdf_f_cols(ds, ['txt'])  # 'П'
                x, y = self.fdf_f_cols(ds, ['x', 'y'])
                ds1 = self.find_cirle(self.df_circle, x, y, 5, 5, 0, 10, radius_lt=2)
                if len(ds1) > 0:
                    df2 = ds1
                    df2['pit'] = name_sh
                    df2['h_name_sh'] = h
                    df2['one_k'] = 'one'

                df_lines = self.find_polyline(self.df_poly, x, y, 5, 5, 0, 10)

                if len(df_lines) > 0:
                    is_line = False
                    for i in range(len(df_lines)):
                        res = df_lines.iloc[i]
                        if res['x1'] == res['x2']:
                            is_line = True
                            break
                    if is_line:
                        df2 = self.find_cirle(self.df_circle, x, y, 5, 5, 0, 180, radius_lt=2)
                        df2['pit'] = name_sh
                        df2['h_name_sh'] = h
                        df2['one_k'] = 'many'

                if df2 is not None:
                    df2['name_sxema'] = name_sxema
                    if self.df_sx_sh_circle is None:
                        self.df_sx_sh_circle = df2.copy()
                    else:
                        self.df_sx_sh_circle = self.df_sx_sh_circle.append(df2)
                        self.df_sx_sh_circle = self.df_sx_sh_circle.reset_index(drop=True)

                    self.df_sx_sh_circle.to_csv(self._get_path_code(prefix='sx_sh_circle', ext='.csv', folder='csv/'), index=False, sep='$')

In [41]:
ad.fdf_filter(ad.df_st_table, ('st', '1'), ('pol', '1'), ('rd', '8'), ('k', '23'))

,st,pol,rd,k,r,c,h
6,1,1,8,23,11,0,3DCE7


In [ ]:
 def _sx_set_pit_contact(self, name_sxema, apart=False):
        if apart:
            self._set_doc(name_sxema)
        df1 = self.df_sx_contacts[self.df_sx_contacts['b1'].isnull()]
        for i in range(len(df1)):
            res = df1.iloc[i]
            h = res['h']
            df_sx_sh_circle = self.fdf_filter(self.df_sx_sh_circle, ('name_sxema', name_sxema))
            df2 = self._find_link_by_sh(df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
            if df2 is not None:
                df3 = df2['df']
                pit = df3['pit'].values[0]

                self.df_sx_contacts = self.fdf_set_val(self.df_sx_contacts, h, 'pit', pit)

In [16]:
df1 = ad.df_sx_contacts[ad.df_sx_contacts['b2'].isnull()]
df1

,loc,st,pol,rd,name,k,h,x,y,group,direction,side,name_sxema,b1,b2,l1,l2,pit
132,1-82,1,8,2,5А,11,2CB2A,7534.03,120.84,1-82;11:12,horizontal,left,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
133,1-82,1,8,2,5А,12,2CB2B,7541.56,120.26,1-82;11:12,horizontal,right,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
138,2-68,2,6,8,І ГП,11,2D0E9,7970.56,128.11,2-68;11:12,horizontal,left,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
140,2-67,2,6,7,ІІ ГП,11,2D0ED,7991.56,118.11,2-67;11:12,horizontal,left,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
142,2-66,2,6,6,1 СП,11,2D0E8,7970.56,108.11,2-66;11:12,horizontal,left,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,1-88,1,8,8,"0,5 А",1,2EA8E,1159.67,-203.74,1-88;2:1,horizontal,right,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
983,1-61,1,6,1,КД1056Б,21,2EAA1,1180.32,-177.73,1-61;22:21,horizontal,right,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
985,1-61,1,6,1,1-61,11,2EA9F,1180.33,-157.74,1-61;12:11,horizontal,right,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
987,1-61,1,6,1,КД1056Б,31,2EAA3,1180.32,-225.73,1-61;32:31,horizontal,right,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN


In [25]:
df1 = ad.fdf_filter(df1, ('name_sxema', ad.circuit_ls_name[0]))

In [27]:
i = 20
res = df1.iloc[i]
h = res['h']
df_sx_sh_circle = ad.fdf_filter(ad.df_sx_sh_circle, ('name_sxema', ad.circuit_ls_name[0]))

In [28]:
df2 = ad._find_link_by_sh(df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])

In [30]:
pit = df2['pit'].values[0]

In [21]:
ad._set_doc(ad.circuit_ls_name[0])

In [20]:
ad.circuit_ls_name[0]

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg'

In [22]:
ad.doc.Name

'2517-АТР.2 л.3 Схемы РЦ Неч..dwg'

In [6]:
nsx = ad.circuit_ls_name[0]
nsx

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg'

In [8]:
df = ad.fdf_filter(ad.df_sx_contacts, ('name_sxema', nsx))
df.head(3)

,loc,st,pol,rd,name,k,h,x,y,group,direction,side,name_sxema,b1,b2,l1,l2,pit
0,2-68,2,6,8,І ГП,82,29AD1,1671.56,140.83,2-68;82:22,vertical,top,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
1,2-68,2,6,8,І ГП,22,29AD4,1671.56,130.81,2-68;82:22,vertical,bottom,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN
2,2-67,2,6,7,ІІ ГП,82,29B02,1671.56,100.83,2-67;82:22,vertical,top,D:\Фабричная\Фабричная\Принципиальные схемы\25...,NaN,NaN,NaN,NaN,NaN


In [9]:
x, y = df['x'].values[0], df['y'].values[0]
x, y

(1671.56, 140.83)

In [30]:
ad._set_doc(nsx)

In [32]:
ss1 = ad.doc.SelectionSets.Add('ss2')
ss1

<win32com.gen_py.AutoCAD 2021 Type Library.IAcadSelectionSet instance at 0x2114752849664>

In [20]:
p1 = ad.a_point(x-50, y-50)
p2 = ad.a_point(x+50, y+50)
p1, p2

(win32com.client.VARIANT(8197, (1621.56, 90.83000000000001, 0)),
 win32com.client.VARIANT(8197, (1721.56, 190.83, 0)))

In [ ]:
ss1.SelectOnScreen()

In [33]:
ss1.Select(1, p1, p2)

In [19]:
ss1.Count

0

In [90]:
ad.circuit_ls_name

['D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.3 Схемы РЦ Неч..dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.4 Схемы РЦ Чет..dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.9 Переезды.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.10 Схемы увязок.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.11 Ограждение.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.12 Загальні схеми.dwg',
 'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.13 Контр.предохр..dwg']

In [116]:
nsx = ad.circuit_ls_name[2]
ad._set_doc(nsx)

In [113]:
# ad._read_mtexts()

In [18]:
s1 = '2Ш-П'
pol, pit = s1.split('Ш-')
pol, pit

('2', 'П')

In [117]:
ad._sx_search_contacts(nsx, True)

In [96]:
nsx

'D:\\Фабричная\\Фабричная\\Принципиальные схемы\\2517-АТР.2 л.13 Контр.предохр..dwg'

In [110]:
ad.fdf_match(ad.df_mtext, r'113')

,h,txt,height,width,x,y,rot,layer,style,color,linetype,linetype_scale,lineweight,material
442,29DD8,113,1.333565,0.0,1594.96,91.28,0.0,PDF2_Text,ISOCPEUR,3,ByLayer,1.0,-1,ByLayer


In [9]:
df = ad.fdf_filter(ad.df_table, ('rows', 21), ('columns', 3))
df

,h,height,width,rows,columns,x,y,layer,style,color,linetype,linetype_scale,lineweight,material
10,46BB8,87.0,50.0,21,3,8208.49,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
11,46BB9,87.0,50.0,21,3,8143.82,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
12,46BBA,87.0,50.0,21,3,8080.65,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
13,46BBB,87.0,50.0,21,3,8017.70,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
14,46BBC,87.0,50.0,21,3,7954.74,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
15,46BBD,87.0,50.0,21,3,7890.07,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
16,46BBE,87.0,50.0,21,3,7786.78,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
17,46BBF,87.0,50.0,21,3,7723.82,264.12,0,Standard,0,ByLayer,1.0,0,ByLayer
18,46BC0,87.0,50.0,21,3,7723.82,147.12,0,Standard,0,ByLayer,1.0,0,ByLayer
19,46BC1,87.0,50.0,21,3,7786.78,147.12,0,Standard,0,ByLayer,1.0,0,ByLayer


In [22]:
ls = []
for i in range(len(df)):
    h = df.iloc[i]['h']
    b1 = ad.get_obj(h)
    name = b1.GetCellValue(0, 1)
    ls.append([name, h])
df1 = pd.DataFrame(ls, columns=['name', 'h'])
df1

,name,h
0,В112,46BB8
1,В111,46BB9
2,В110,46BBA
3,В19,46BBB
4,В18,46BBC
5,В17,46BBD
6,В16,46BBE
7,В15,46BBF
8,В25,46BC0
9,В26,46BC1


In [10]:
b1 = ad.get_obj(df.iloc[0]['h'])

In [11]:
b1.GetCellValue(0, 1)

'В112'

In [12]:
b1.SetCellValue(2, 0, 'test')

In [15]:
ds_v_row_col = {
    '1': (1, 0),
    '2': (1, 2),
    '3': (3, 0),
    '4': (3, 2),
    '5': (5, 0),
    '6': (5, 2),
    '7': (7, 0),
    '8': (7, 2),
    '9': (9, 0),
    '10': (9, 2),
    '11': (11, 0),
    '12': (11, 2),
    '13': (13, 0),
    '14': (13, 2),
    '15': (15, 0),
    '16': (15, 2),
    '17': (17, 0),
    '18': (17, 2),
    '19': (19, 0),
    '20': (19, 2),
}

In [29]:
s1 = '3-В112-14'
st, v1, s1 = s1.split('-')
k1 = ds_v_row_col[s1]
st, v1, s1, k1

('3', 'В112', '14', (13, 2))

In [ ]:
s1 = 'В112-14'
v1, s1 = s1.split('-')
k1 = ds_v_row_col[s1]
v1, s1, k1

In [30]:
s1 = '3-В112-14'
s2 = s1.split('-')
s2

['3', 'В112', '14']

In [36]:
st=None
s1 = 'В112-15'
s2 = s1.split('-')
if len(s2) == 3:
    st, v1, s1 = s2
    k1 = ds_v_row_col[s1]
elif len(s2) == 2:
    v1, s1 = s2
    k1 = ds_v_row_col[s1]
else:
    print('error')
st, v1, s1, k1

(None, 'В112', '15', (15, 0))

In [23]:
def find_nearest(ls, value):
    ls = np.asarray(ls)
    idx = (np.abs(ls - value)).argmin()
    return ls[idx]

In [29]:
find_nearest([2,4,6,8,10], 10)

10

In [17]:
b1.SetCellValue(k1[0], k1[1], 'test')

In [18]:
b1.SetCellValue(k1[0]+1, k1[1], 'test1')

In [185]:
loc = '3-68'
k = '72'
val = 'В15-8'
ad.st_set_k(loc, k, val)

In [ ]:
df1 = ad.df_st_table
df1.head(3)

In [ ]:
loc = '3-68'
k = '72'
st, p, r = ad.get_loc(loc)
st, p, r, k

In [ ]:
df1 = ad.fdf_filter(ad.df_st_table, ('st', st), ('pol', p), ('rd', r), ('k', k))
df1

In [ ]:
b1 = ad.docs_open[ad.montag_ds_name[st]]
b1

In [ ]:
b2 = ad.get_obj(df1.iloc[0]['h'], b1)
s1 = b2.GetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c']+1)
s2 = b2.GetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c']+2)
print(s1, s2)
b2.SetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c'] + 1, '123')

In [ ]:
def st_set_k(loc, k, val):
    if type(loc) == str:
        st, p, r = ad.get_loc(loc)
    else:
        st, p, r = loc
    df1 = ad.fdf_filter(ad.df_st_table, ('st', st), ('pol', p), ('rd', r), ('k', k))
    if len(df1) > 0:
        b1 = ad.docs_open[ad.montag_ds_name[st]]
        b2 = ad.get_obj(df1.iloc[0]['h'], b1)
        b2.SetCellValue(df1.iloc[0]['r'], df1.iloc[0]['c'] + 1, val)
    else:
        print('Не найден объект в таблице')

In [ ]:
loc = '3-68'
k = '71'
st_set_k(loc, k, '124')

In [ ]:
ad._create_df_table_montag()

In [ ]:
df1 = ad.df_mtext
df1.head(3)

In [ ]:
df2 = ad.df_match(df1, r'^[1-3]-\d{2,3}')
df2.head(3)

In [ ]:
lsh = df2['h'].tolist()


In [ ]:
def fdf_h(df, h):
    df = df[df['h']==h]
    return df

def fdf_h_cols(df, h, cols):
    df = df[df['h']==h][cols]
    if len(cols) == 1:
        return df.iloc[0][cols[0]]
    if len(df) == 0:
        return None
    return df.iloc[0].tolist()

def fdf_f(df):
    if df is None:
        return None
    if len(df) == 0:
        return None
    return df.iloc[0]

def fdf_f_cols(df, cols):
    if df is None:
        return None
    if len(df) == 0:
        return None
    if len(cols) == 1:
        return df.iloc[0][cols[0]]
    return df.iloc[0][cols].tolist()

def fdf_f_h(df):
    if df is None:
        return None
    if len(df) == 0:
        return None
    return df.iloc[0]['h']

def fdf_filter(df, col1_val1, col2_val2=None, col3_val3=None, col4_val4=None, col5_val5=None, col6_val6=None, col7_val7=None, col8_val8=None, col9_val9=None, col10_val10=None):
        col1, val1 = col1_val1
        df1 = df[df[col1] == val1]
        if col2_val2 is not None:
            col2, val2 = col2_val2
            df1 = df1[df1[col2] == val2]

        if col3_val3 is not None:
            col3, val3 = col3_val3
            df1 = df1[df1[col3] == val3]

        if col4_val4 is not None:
            col4, val4 = col4_val4
            df1 = df1[df1[col4] == val4]

        if col5_val5 is not None:
            col5, val5 = col5_val5
            df1 = df1[df1[col5] == val5]

        if col6_val6 is not None:
            col6, val6 = col6_val6
            df1 = df1[df1[col6] == val6]

        if col7_val7 is not None:
            col7, val7 = col7_val7
            df1 = df1[df1[col7] == val7]

        if col8_val8 is not None:
            col8, val8 = col8_val8
            df1 = df1[df1[col8] == val8]

        if col9_val9 is not None:
            col9, val9 = col9_val9
            df1 = df1[df1[col9] == val9]

        if col10_val10 is not None:
            col10, val10 = col10_val10
            df1 = df1[df1[col10] == val10]

        return df1

def _direction_contact(df):
        if len(df) > 1:
            c1 = df.iloc[0]['txt']
            c2 = df.iloc[1]['txt']
            x1 = df.iloc[0]['x']
            x2 = df.iloc[1]['x']
            y1 = df.iloc[0]['y']
            y2 = df.iloc[1]['y']

            dx = abs(x1 - x2)
            dy = abs(y1 - y2)

            if dx > 2:
                if x1 < x2:
                    return {c1: ['horizontal', 'left'], c2: ['horizontal', 'right']}
                else:
                    return {c1: ['horizontal', 'right'], c2: ['horizontal', 'left']}
            if dy > 2:
                if y1 > y2:
                    return {c1: ['vertical', 'top'], c2: ['vertical', 'bottom']}
                else:
                    return {c1: ['vertical', 'bottom'], c2: ['vertical', 'top']}

def get_loc(loc):
        ls = loc.split('-')
        st = ls[0]
        pr = ls[1]
        if len(pr) == 2:
            p = pr[0]
            r = pr[1]
        else:
            p = pr[0:2]
            r = pr[2]
        return st, p, r

In [ ]:
fdf_h_cols(df1, lsh[0], ['txt', 'x', 'y']), fdf_h_cols(df1, lsh[0], ['txt'])

In [ ]:
fdf_f(df1)

In [ ]:
fdf_f_h(df1)

In [ ]:

df = ad.df_mtext
df_dev = ad.df_match(df, r'^[1-3]-\d{2,3}')
lsh = df_dev['h'].tolist()

ls = []

for h in lsh:
    loc = fdf_h_cols(df, h, ['txt'])
    x, y = fdf_h_cols(df, h, ['x', 'y'])

    df_name = ad.find_text(df1, x, y, 8, 8, 0, 5, lst_exclude=ad.ls_contact, h=h)
    if len(df_name) == 0:
        continue
    h_name = fdf_f_h(df_name)
    t_name = fdf_f_cols(df_name, ['txt'])
    x, y = fdf_f_cols(df_name, ['x', 'y'])
    df_circle = ad.find_cirle(ad.df_circle, x, y, 20, 20, 8, 9, radius_gt=3)
    if len(df_circle) > 0:
        h_c= fdf_f_h(df_circle)
        x, y = fdf_f_cols(df_circle, ['x', 'y'])
        df_k = ad.find_text(ad.df_mtext, x, y, 10, 10, 10, 10, lst_filter=ad.ls_contact, h=h_name)
    else:
        df_k = ad.find_text(ad.df_mtext, x, y, 20, 20, 0, 12, lst_filter=ad.ls_contact, h=h_name)

    if len(df_k) == 0:
        continue

    if df_k['txt'].tolist() == ['1', '2', '3', '4']:
        df_k = df_k[df_k['txt'].isin(['2', '4'])]
    ds = _direction_contact(df_k)
    group = loc + ';' + ':'.join(df_k['txt'].values.tolist())

    for j in range(len(df_k)):
        k = df_k.iloc[j]['txt']
        t_ref = df_k.iloc[j]['h']
        t_x = df_k.iloc[j]['x']
        t_y = df_k.iloc[j]['y']
        st, p, r = get_loc(loc)
        ls.append([loc, st, p, r, t_name, k, t_ref, t_x, t_y, group, ds[k][0],
                                   ds[k][1]])


df_sx_contacts = pd.DataFrame(ls, columns=['loc', 'st', 'p', 'r', 'name', 'contact', 'h', 'x', 'y', 'group', 'direction',
                                        'side'])

df_sx_contacts[['b1', 'b2', 'l1', 'l2', 'pit']] = None





In [ ]:
df = ad.df_table
df

In [ ]:
b1 = ad.get_obj(fdf_f_h(df))

In [ ]:
xy = b1.InsertionPoint
xy

In [ ]:
df = ad.find_text(ad.df_text, xy[0], xy[1], -300, 390, -245, 290)
df

In [ ]:
df = ad.df_match(df, r'^Полка')
df

In [ ]:
s1 = fdf_f_cols(df, ['txt'])
s1

In [ ]:
s1 = s1.split(' ')[-1]
s1

In [ ]:
xy = b1.InsertionPoint
df = ad.find_text(ad.df_text, xy[0], xy[1], -300, 390, -245, 290)
df = ad.df_match(df, r'^Полка')
s1 = fdf_f_cols(df, ['txt'])
if s1 is not None:
    s1 = s1.split(' ')[-1]
s1

In [ ]:
df

In [ ]:
df = ad.df_match(df, r'^Полка')

In [ ]:
fdf_f_cols(df, ['txt'])

In [ ]:
b1.Rows, b1.Columns

In [ ]:
for st, name_sx in ad.montag_ds_name.items():
    print(st, name_sx)

In [ ]:
ad.docs_open[name_sx]

In [ ]:
ls = []
df = pd.DataFrame(ls, columns=['st', 'pol', 'rd', 'k', 'r', 'c', 'h'])

for st, name_sx in ad.montag_ds_name.items:
    doc = ad.docs_open[name_sx]
    for h in ad.df_table['h'].tolist():
        b1 = ad.get_obj(h)
        xy = b1.InsertionPoint
        df1 = ad.find_text(ad.df_text, xy[0], xy[1], -300, 390, -245, 290)
        df1 = ad.df_match(df1, r'^Полка')
        polka = fdf_f_cols(df1, ['txt'])

        if polka is not None:

            polka = int(polka.split(' ')[-1])

            for c in range(b1.Columns):
                ls1 = []
                v1 = b1.GetCellValue(0, c)
                if v1 is None:
                    continue
                for r in range(2, b1.Rows):
                    v2 = b1.GetCellValue(r, c)
                    if v2 is None:
                        continue
                    ls1.append([st, polka, v1, v2, r, c, h])
                # ls.append(ls1)
                df = pd.concat([df, pd.DataFrame(ls1, columns=['st', 'pol', 'rd', 'k', 'r', 'c', 'h'])])

    df = df.reset_index(drop=True)
    df = df.astype({'st': 'str', 'pol': 'str', 'rd': 'str', 'k': 'str'})

df

In [ ]:
df2 = fdf_filter(df, ('st', 3), ('pol', 7), ('rd', 5), ('k', 31))
df2

In [ ]:
b2 = ad.get_obj(df2.iloc[0]['h'])
s1 = b2.GetCellValue(df2.iloc[0]['r'], df2.iloc[0]['c'])
print(s1)
b2.SetCellValue(df2.iloc[0]['r'], df2.iloc[0]['c'] + 1, '123')

In [ ]:
loc = '3-74'
kont = '52'


In [ ]:
get_loc(loc)

In [ ]:
df = df.astype({'st': 'str', 'pol': 'str', 'rd': 'str', 'k': 'str'})
df = df.reset_index(drop=True)


In [ ]:
df

In [ ]:
df[(df['st'] == )]

In [ ]:
df.info()

In [ ]:
rc = df[(df['rd'] == '8') & (df['k'] == '32')][['r', 'c']].values[0].tolist()
rc

In [ ]:
b1.SetCellValue(rc[0]+3, rc[1] + 1, '65-42')
b1.SetCellValue(rc[0]+3, rc[1] + 2, 'В15-20')

In [13]:
ls = []
for i in range(b1.Rows):
    ls1 = []
    for j in range(b1.Columns):
        ls1.append(b1.GetCellValue(i, j))
    ls.append(ls1)

df = pd.DataFrame(ls, columns=[8, 18,28, 7, 17, 27, 6, 16, 26, 5, 15, 25, 4, 14, 24, 3, 13, 23, 2, 12, 22, 1, 11, 21])
df

,8,18,28,7,17,27,6,16,26,5,...,24,3,13,23,2,12,22,1,11,21
0,8,None,None,7,None,None,6,None,None,5,...,None,3,None,None,2,None,None,1,None,None
1,№к.,116СП,None,№к.,162/166П,None,№к.,166П,None,№к.,...,None,№к.,314СП,None,№к.,2БП,None,№к.,1БП,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,12,верх,None,12,верх,None,12,верх,None,12,...,None,12,верх,None,12,верх,None,12,верх,None
7,11,Ш,П,11,Ш,П,11,Ш,П,11,...,П,11,Ш,П,11,Ш,П,11,Ш,П
8,13,None,None,13,None,None,13,None,None,13,...,None,13,None,None,13,None,None,13,None,None
9,22,верх 2Р,None,22,верх 2Р,None,22,верх 2Р,None,22,...,None,22,верх 2Р,None,22,верх 2Р,None,22,верх 2Р,None


In [ ]:
r = 7
r1 = r +10
r2 = r + 20

In [ ]:
row = df[df[r] == 42].index.values[0]
row

In [ ]:
b1.SetCellValue(r2-20, row+1, 'test')

In [ ]:
b1.SetCellValue(0, 0, 'test')

In [ ]:
ad.close_docs()

In [ ]:
ad._read_ent_type()

In [ ]:
ad.df_ent_type['entity_name'].value_counts()

In [ ]:
# b1 = ad.get_obj('2E5AB')
# b2 = ad.get_obj('2DFC0')
# b3 = ad.get_obj('30A13')
# b4 = ad.get_obj('2DFA8')
b5 = ad.get_obj('408A6')

In [ ]:
# ad.df_ent_type[ad.df_ent_type['entity_name']=='AcDbMText']
# ad.df_ent_type[ad.df_ent_type['entity_name']=='AcDbText']
ad.df_ent_type[ad.df_ent_type['entity_name']=='AcDbCircle']

In [ ]:
ad.read_polylines()

In [ ]:
acad.load_polylines()

In [ ]:
acad.df_poly.head(3)

In [ ]:
b1 = acad.get_obj(acad.df_poly['h'].values[0])

In [ ]:
s1 = 'polylines$' + acad.doc.FullName.replace('\\', '$') + '.csv'
s1

In [ ]:
df = acad.get_docs()
df

In [ ]:
acad.get_path_to_csv()

In [ ]:
acad.get_path_to_csv('polylines')

In [ ]:
acad.read_polylines()

In [ ]:
df.to_csv('csv/docs.csv', index=False, sep='$')

In [ ]:
acad.read_ent_type()

In [ ]:
df = acad.df_ent_type

df1 = df[df['entity_name']=='AcDbMText']
df1

In [ ]:
t = df['entity_name'].value_counts()
t

In [ ]:
t.index.tolist()

In [ ]:
h1 = df[df['entity_name']=='AcDbPolyline']['handle'].values[0]
h1

In [ ]:
b1 = acad.get_obj(h1)

In [ ]:
def read_polyline(df):
    df2 = df[df['entity_name']=='AcDbPolyline']

    ls = []
    for h in df2['handle'].values:
        b1 = acad.get_obj(h)
        d = {
            'h': h,
            'length': b1.Length,
            'x1': b1.Coordinates[0],
            'y1': b1.Coordinates[1],
            'x2': b1.Coordinates[-2],
            'y2': b1.Coordinates[-1],
            'points': b1.Coordinates,
            'layer': b1.Layer,
            'color': b1.Color,
            'linetype': b1.Linetype,
            'closed': b1.Closed,
            'area': b1.Area,
            'linetype_scale': b1.LinetypeScale,
            'lineweight': b1.Lineweight,
            'material': b1.Material,
            'thickness': b1.Thickness,
        }
        ls.append(d)

    df3 = pd.DataFrame(ls)
    return df3

In [46]:
df3.to_csv('polyline.csv', index=False, sep='$')

In [48]:
df4 = pd.read_csv('polyline.csv', sep='$')

In [49]:
p1 = df4['points'].values[0]

In [54]:
ls1 = p1.replace('(', '').replace(')', '').split(',')

In [55]:
ls1 = [float(x) for x in ls1]
ls1

[1045.5544423979388,
 -80.27062376316371,
 1048.0545834305792,
 -80.27062376316371,
 1129.0546373236336,
 -80.27062376316371,
 1129.0546373236336,
 -88.27058117774595]

In [ ]:
def f1(handle):
    obj = acad.get_obj(handle)
    ser = pd.Series([obj.Handle, obj.TextString, obj.InsertionPoint[0], obj.InsertionPoint[1], obj.Height, obj.Width, obj.Rotation, obj.StyleName, obj.Layer, obj.color])
    return ser


df1[['handle', 'str', 'x', 'y','height', 'width', 'rotation', 'style', 'layer', 'color']] = df1['handle'].apply(f1)
df1

In [32]:
ob2 = acad.get_obj(df1.loc[7796]['handle'])
ob2.TextString

'XK21:5'

In [34]:
df1.to_csv('mtext.csv', index=False)

In [35]:
df2 = pd.read_csv('mtext.csv')

In [131]:
df1['txt'].value_counts()

1         409
2         394
4         370
3         359
0,75      162
         ... 
W1239       1
XK71:9      1
XK71:8      1
W1236       1
FSP233      1
Name: txt, Length: 1333, dtype: int64

In [ ]:
# OfficeEvents - test/demonstrate events with Word and Excel.
from win32com.client import DispatchWithEvents, Dispatch
import msvcrt, pythoncom
import time, sys
import types

import threading
stopEvent = threading.Event()

def TestExcel():
    class ExcelEvents:
        def OnNewWorkbook(self, wb):
            if type(wb) != types.InstanceType:
                raise RuntimeError, "The transformer doesnt appear to have translated this for us!"
            self.seen_events["OnNewWorkbook"] = None
        def OnWindowActivate(self, wb, wn):
            if type(wb) != types.InstanceType or type(wn) != types.InstanceType:
                raise RuntimeError, "The transformer doesnt appear to have translated this for us!"
            self.seen_events["OnWindowActivate"] = None
        def OnWindowDeactivate(self, wb, wn):
            self.seen_events["OnWindowDeactivate"] = None
        def OnSheetDeactivate(self, sh):
            self.seen_events["OnSheetDeactivate"] = None
        def OnSheetBeforeDoubleClick(self, Sh, Target, Cancel):
            if Target.Column % 2 == 0:
                print "You can double-click there..."
            else:
                print "You can not double-click there..."
            # This function is a void, so the result ends up in
            # the only ByRef - Cancel.
                return 1

    class WorkbookEvents:
        def OnActivate(self):
            print "workbook OnActivate"
        def OnBeforeRightClick(self, Target, Cancel):
            print "It's a Worksheet Event"

    e = DispatchWithEvents("Excel.Application", ExcelEvents)
    e.seen_events = {}
    e.Visible=1
    book = e.Workbooks.Add()
    book = DispatchWithEvents(book, WorkbookEvents)
    print "Have book", book
#    sheet = e.Worksheets(1)
#    sheet = DispatchWithEvents(sheet, WorksheetEvents)

    print "Double-click in a few of the Excel cells..."
    print "Press any key when finished with Excel, or wait 10 seconds..."
    if not _WaitForFinish(e, 10):
        e.Quit()
    if not _CheckSeenEvents(e, ["OnNewWorkbook", "OnWindowActivate"]):
        sys.exit(1)

def TestWord():
    class WordEvents:
        def OnDocumentChange(self):
            self.seen_events["OnDocumentChange"] = None
        def OnWindowActivate(self, doc, wn):
            self.seen_events["OnWindowActivate"] = None
        def OnQuit(self):
            self.seen_events["OnQuit"] = None
            stopEvent.set()

    w = DispatchWithEvents("Word.Application", WordEvents)
    w.seen_events = {}
    w.Visible = 1
    w.Documents.Add()
    print "Press any key when finished with Word, or wait 10 seconds..."
    if not _WaitForFinish(w, 10):
        w.Quit()
    if not _CheckSeenEvents(w, ["OnDocumentChange", "OnWindowActivate"]):
        sys.exit(1)

def _WaitForFinish(ob, timeout):
    end = time.time() + timeout
    while 1:
        if msvcrt.kbhit():
            msvcrt.getch()
            break
        pythoncom.PumpWaitingMessages()
        stopEvent.wait(.2)
        if stopEvent.isSet():
            stopEvent.clear()
            break
        try:
            if not ob.Visible:
                # Gone invisible - we need to pretend we timed
                # out, so the app is quit.
                return 0
        except pythoncom.com_error:
            # Excel is busy (eg, editing the cell) - ignore
            pass
        if time.time() > end:
            return 0
    return 1

def _CheckSeenEvents(o, events):
    rc = 1
    for e in events:
        if not o.seen_events.has_key(e):
            print "ERROR: Expected event did not trigger", e
            rc = 0
    return rc

def test():
    import sys
    if "noword" not in sys.argv[1:]:
        TestWord()
    if "noexcel" not in sys.argv[1:]:
        TestExcel()
    print "Word and Excel event tests passed."

if __name__=='__main__':
    test()